In [5]:
from bs4 import BeautifulSoup as soup
import pandas as pd

import requests
import time
import random

def wait ():
    sleep_time = {'min': 0.5, 'max': 1}
    time.sleep(sleep_time["min"]+(random.randrange(0, 10)*0.1*(sleep_time["max"]-sleep_time["min"])))

url = "https://nn.hh.ru/search/vacancy"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
    "Accept": "*/*",
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive"
}

params = {
    "clusters": False,
    "ored_clusters": False,
    "enable_snippets": False,
    "salary": None,
    "st": "searchVacancy",
    "text": "golang",
    "page": 0
}

vacancys_data = []
vacancys_on_page = True 

while vacancys_on_page:
    
    response = requests.get(url, headers = headers, params = params)
    response_parsed = soup(response.text, 'html.parser')
    
    # ищем блоки с вакансиями и выдергиваем url
    vacancys_on_page = response_parsed.select("div[class=vacancy-serp-item]")  
    vacancys_on_page = (vacancys_on_page if len(vacancys_on_page or [])>0 else False)
    
    # если нашли вакансии - составляем под них структуры с данными
    if vacancys_on_page:
        for vacancy_block in vacancys_on_page:
            link_container = vacancy_block.select("a[class='bloko-link']")
            vacancys_data.append({"url": link_container[0]['href']})
    
    print("page ", params['page'], ", count: ", len(vacancys_on_page or []) )    
    params['page'] += 1
    
    # симулируем
    wait()

page  0 , count:  18
page  1 , count:  20
page  2 , count:  20
page  3 , count:  20
page  4 , count:  20
page  5 , count:  20
page  6 , count:  20
page  7 , count:  20
page  8 , count:  20
page  9 , count:  20
page  10 , count:  20
page  11 , count:  20
page  12 , count:  20
page  13 , count:  20
page  14 , count:  20
page  15 , count:  20
page  16 , count:  20
page  17 , count:  20
page  18 , count:  20
page  19 , count:  20
page  20 , count:  20
page  21 , count:  20
page  22 , count:  20
page  23 , count:  20
page  24 , count:  20
page  25 , count:  20
page  26 , count:  20
page  27 , count:  20
page  28 , count:  0


In [6]:
# разбираем страницу вакансии
counter = 1
for data in vacancys_data:
    
    response = requests.get(data['url'], headers = headers)
    response_parsed = soup(response.text, 'html.parser')    
   
    # вытаскиваем заголовок
    
    title = response_parsed.select("h1[data-qa = 'vacancy-title']")
    data["title"] = str(title[0].get_text()) if len(title)>0 else False
                        
    # зарплата бля(
    
    salary_span = response_parsed.select("span[data-qa = 'bloko-header-2'][class = 'bloko-header-2 bloko-header-2_lite']")
    
    #print(counter," of ",len(vacancys_data)," : ", salary_span.get_text())
    counter += 1
    
    data["salary"] = str(salary_span[0].get_text()) if len(salary_span)>0 else False
    
    # название компании
    
    company_name_container = response_parsed.select("a[data-qa = 'vacancy-company-name'][class = 'vacancy-company-name'] > span[data-qa = 'bloko-header-2']")
    
    data["company_name"] = company_name_container[0].get_text() if len(company_name_container)>0 else False
    
    # проверенный работодатель
    
    verified_company_link = response_parsed.select("div[class = 'vacancy-company-badge'] > a[class = 'bloko-link'][href = 'https://feedback.hh.ru/article/details/id/5951']")
    data["company_verified"] = True if len(verified_company_link)>=1 else False
    
    # адрес 
    
    full_adress = response_parsed.select("span[data-qa = 'vacancy-view-raw-address']") 
    city_or_smth = response_parsed.select("p[data-qa = 'vacancy-view-location']")
    
    city_or_smth = city_or_smth[0] if len(city_or_smth)>0 else False
    full_adress = full_adress[0] if len(full_adress)>0 else False
        
    data["adress"] = ""
    
    if city_or_smth or full_adress:
        data["adress"] = (city_or_smth or full_adress).get_text()
        
    # режим на объекте
    employment_mode = response_parsed.select("p[data-qa = 'vacancy-view-employment-mode']") 
    data["employment_mode"] = employment_mode[0].get_text() if len(employment_mode)>0 else False
    
    # опыт работы 
    experience = response_parsed.select("span[data-qa = 'vacancy-experience']") 
    data["experience"] = experience[0].get_text() if len(experience)>0 else False
    
    # parttime
    parttime = response_parsed.select("p[data-qa = 'vacancy-view-parttime-options']") 
    data["parttime"] = parttime[0].get_text() if len(parttime)>0 else False

    # accept-temporary
    accept_temporary = response_parsed.select("p[data-qa = 'vacancy-view-accept-temporary']") 
    data["accept_temporary"] = accept_temporary[0].get_text() if len(accept_temporary)>0 else False

    # full_description
    description = response_parsed.select("div[class = 'vacancy-section']") 
    data["description"] = description[0].get_text() if len(description)>0 else False
  
    # дата
    date = response_parsed.select("p[class = 'vacancy-creation-time']") 
    data["creation-time"] = date[0].get_text() if len(date)>0 else False
    
    # тэги 
    tags = response_parsed.select("div[class = 'bloko-tag-list'] > div") 
    tags = tags[0] if len(tags)>0 else False
    
    tag_list = []
    if tags:
        for tag in tags.children:
            tag_list.append(tag.get_text())
    
    data["tag"] = tag_list
           
    print(data["accept_temporary"], data["company_name"])
    
    # симулируем
    wait()    



False InterWeb OÜ
False ТТ ДЕВЕЛОПМЕНТ
False Голик Марина Сергеевна
False ООО МАКСБИТСОЛЮШЕН
Возможно временное оформление: договор услуг, подряда, ГПХ, самозанятые, ИП Kvando Technologies
False B2Broker Киев
False ООО Софтвайс
False inDriver
False ЮнгЭдс
Возможно временное оформление: договор услуг, подряда, ГПХ, самозанятые, ИП ШтрафовНет.ру
False ООО Софтвайс
Возможно временное оформление: договор услуг, подряда, ГПХ, самозанятые, ИП ООО Селфсек
False ООО ЮТЕКА
False Skycoin
False Точка
False Digital Kitchen (ООО Цифровая кухня)
False B2Broker Киев
False ООО Медиа-тел
Возможно временное оформление: договор услуг, подряда, ГПХ, самозанятые, ИП ЦифровойТы
False inDriver
False B2Broker Москва
Возможно временное оформление: договор услуг, подряда, ГПХ, самозанятые, ИП WILDBERRIES
False ООО Расчетные системы
False Beeline, ТМ
False ООО Сима-ленд
False inDriver
False ООО Evrius
False ООО Смарт Текнолоджис
False B2Broker Москва
False Gaijin Games
False ООО Смарт Текнолоджис
False OUTSIDE D

False ГК Орион телеком
Возможно временное оформление: договор услуг, подряда, ГПХ, самозанятые, ИП ООО Арум Капитал
False Nexters
False Prequel
False Nexters
False ГК Орион телеком
False РОССТ, РА
False ООО ИММО
False ГК Орион телеком
False Hunt For You
False Ozon Информационные технологии
False Hunt For You
False Ozon Информационные технологии
False Faberlic
False Ozon Информационные технологии
False AEON
False Электронная торговая площадка Газпромбанка
False Ozon Информационные технологии
False Гарант
False Ozon Информационные технологии
False Marfatech
False РЕЗОНИТ
False Корпорация ЭЛАР
False Авито. Продажи
False Group-IB
False ООО Нокиа Солюшнз Энд Нетворкс
False FIX
False Ассоциация IPChain
False ООО Константа
False Туту.ру
False MERLION
False QTECH
False Quadcode
False ЗАЗЕКС
False ООО Вебинар Технологии
False ООО Ростелеком Информационные Технологии
False SberCloud
False Quiet Media
False Авито. Продажи
False ivi
False Nexters
False Nexters
False Netcracker Technology Corp.
Fal

In [7]:
! git add *
! git commit 
! git push 

The file will have its original line endings in your working directory
The file will have its original line endings in your working directory


[main 0ebc080] damn, this shit works
 2 files changed, 845 insertions(+), 73 deletions(-)


To https://github.com/russianSlave/hh_ru_parser.git
   c0241ba..0ebc080  main -> main
